# Example 4-2: Finding Observation Values
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 279-280

This notebook demonstrates converting between various observation types from surveillance and tracking systems.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment and `pandas` for visual data display:

In [1]:
!pip install valladopy pandas

Import libraries and `valladopy` modules:

In [2]:
import numpy as np
import pandas as pd

import valladopy.astro.twobody.frame_conversions as fc
from valladopy.astro.time.data import iau80in
from valladopy.astro.time.frame_conversions import ecef2eci, eci2ecef
from valladopy.astro.twobody.utils import site
from valladopy.mathtime.julian_date import convtime

## Problem Definition
---

GIVEN: &ensp; Neptune on May 14, 1994, r = 29.664361 AU (1 AU = 149,597,870 km)<br>
&emsp;&emsp;&emsp;&emsp; A site at the U.S. Air Force Academy with:
                         $\phi_{gd} = 39.007^{\circ}$, $\lambda = -104.883^{\circ}$, $h_{ellp} = 2194.56\ \text{m}$,
                         and an epoch time of 13:11:20.59856 UTC<br>
FIND: &emsp;&ensp;$\vec{r}$, $\vec{v}$, $\alpha$, $\delta$, $\alpha_t$, $\delta_t$, $\beta$, $el$, $\phi_{ecliptic}$, $\lambda_{ecliptic}$, and rates

In [3]:
# Date
year = 1994
month = 5
day = 14
hour = 13
minute = 11
second = 20.59856
timezone = 0  # hours offset from UTC

# Geo coordinates
latgd = np.radians(39.007)  # geodetic latitude, rad
lon = np.radians(-104.883)  # longitude, rad
alt = 2.19456               # altitude, km

## Solution
---

Start by getting $\Delta AT$, from the *Astronomic Almanac* (or from USNO's historical list [here](https://maia.usno.navy.mil/ser7/tai-utc.dat)):

$$
\Delta AT = 32.0^{\text{S}}
$$

In [4]:
dat = 32  # seconds

We first find the geocentric right ascension and declination for the given time. We can retreive this from the [NASA JPL Horizons system](https://ssd.jpl.nasa.gov/horizons) using their public API. This process is outlined in the [JPL Horizons Queries notebook](JPL_Horizons_Queries.ipynb) (see **Right Ascension, Declination, and Range** section), but for this exercise, we will use the RA/dec angles, RA/dec angle rates, range, and range rate provided in the textbook example:

In [5]:
rr = 4437725220.51                   # range, km
drr = -25.53033094                   # range rate, km/s
rtasc = np.radians(294.9891458)      # right ascension, rad
decl = np.radians(-20.8234944)       # declination, rad
drtasc = np.radians(-0.00000012244)  # right ascension rate, rad/s
ddecl = np.radians(-0.00000001794)   # declination rate, rad/s

The Earth-centered inertial (ECI) state vectors can be determined by **Equation 4-1** and **Equation 4-2**:

$$
\vec{r}_{ECI} = \begin{bmatrix}
r\cos(\delta) \cos(\alpha) \\
r\cos(\delta) \sin(\alpha) \\
r\sin(\delta) 
\end{bmatrix}
$$

$$
\vec{v}_{ECI} = \begin{bmatrix}
\dot{r}\cos(\delta)\cos(\alpha) - r\sin(\delta)\cos(\alpha)\dot{\delta} - r\cos(\delta)\sin(\alpha)\dot{\alpha}  \\
\dot{r}\cos(\delta)\sin(\alpha) - r\sin(\delta)\sin(\alpha)\dot{\delta} + r\cos(\delta)\cos(\alpha)\dot{\alpha}  \\
\dot{r}\sin(\delta) + r\cos(\delta)\dot{\delta}
\end{bmatrix}
$$

We can accomplish this by calling the `radec2rv` frame conversion routine:

In [6]:
reci, veci = fc.radec2rv(rr, rtasc, decl, drr, drtasc, ddecl)

print(f'reci:\t{reci}\tkm')
print(f'veci:\t{veci}\t\tkm/s')

reci:	[ 1.75224621e+09 -3.75956344e+09 -1.57756810e+09]	km
veci:	[-18.32349705  18.3320495    7.77704123]		km/s


**Algorithm 25** covers the reverse process of converting the ECI state vectors to geocentric right ascension and declination values.

To get the topocentric right ascension and declination, we first need to find the site vector in Earth-fixed coordinates. From **Equation 7-1**:

$$
\vec{r}_{siteECEF} = \begin{bmatrix}
r_{\delta}\cos(\lambda) \\
r_{\delta}\sin(\lambda) \\
r_K 
\end{bmatrix}
= \begin{bmatrix}
(C_{\oplus} + h_{ellp}) \cos(\phi_{gd}) \cos(\lambda) \\
(C_{\oplus} + h_{ellp}) \cos(\phi_{gd}) \sin(\lambda)  \\
(S_{\oplus} + h_{ellp}) \sin(\phi_{gd})
\end{bmatrix}
$$

and:
$$
\vec{v}_{siteECEF} = [0, 0, 0]
$$

We can call the `site` routine to get these vectors:

In [7]:
rsecef, vsecef = site(latgd, lon, alt)

print('ITRF (ECEF) site vectors:')
print(f'r_site:\t{rsecef}\tkm')
print(f'v_site:\t{vsecef}\t\t\t\t\tkm/s')

ITRF (ECEF) site vectors:
r_site:	[-1275.12327899 -4797.99417809  3994.30177136]	km
v_site:	[0. 0. 0.]					km/s


We now need to convert the site vectors from the fixed frame (ECEF) to the inertial frame (ECI). We first need to obtain the nutation matrices for the reduction calculations. The `iau80in` routine requires the nutation data to be present in a given data directory.

Replace the following data directory definition with your preferred location. We will use the relative path from this notebook, assuming the same structure as [repository](https://github.com/CelesTrak/fundamentals-of-astrodynamics) to reach the `datalib` directory. The file name is hardcoded for now but is planned to be flexible in the future — make sure these are included in your data directory.

In [8]:
data_dir = "../../../datalib/"  # relative data path

iau80arr = iau80in(data_dir)

print(f'IAU 80 nutation matrices keys:\n{iau80arr.__dict__.keys()}\n')

IAU 80 nutation matrices keys:
dict_keys(['iar80', 'rar80'])



Next, convert UTC to various time systems using the `convtime` routine (see [Example 3-7](Example_3-7.ipynb) for more details on this process), and for this exercise we will set `dut1` to `0`. We just need the Julian centuries of TT and the Julian date of UT1 for the final transformation:

In [9]:
dut1 = 0
_, _, jdut1, jdut1frac, _, _, _, ttt, *_ = convtime(year, month, day, hour, minute, second, timezone, dut1, dat)

print(f'Julian centuries of TT:\t{ttt}')

Julian centuries of TT:	-0.05634359242066914


Convert the site vectors from the fixed frame (ECEF) to the inertial frame (ECI) using the `ecef2eci` routine, setting the ECEF acceleration and EOP values to zeroes:

In [10]:
# Set dummy values for acceleration and EOP parameters
asecef = [0, 0, 0]                    # dummy acceleration, km/s²
lod, xp, yp, ddpsi, ddeps = (0,) * 5  # EOP parameters

# Convert site vectors from ECEF to ECI
rseci, vseci, _ = ecef2eci(rsecef, vsecef, asecef, ttt, jdut1+jdut1frac, lod, xp, yp, ddpsi, ddeps, iau80arr)

print('GCRF (ECI) site vectors:')
print(f'r_site:\t{rseci}\tkm')
print(f'v_site:\t{vseci}\tkm/s')

GCRF (ECI) site vectors:
r_site:	[ 4068.21275892 -2842.52873922  3996.34951739]	km
v_site:	[2.07272424e-01 2.96810958e-01 1.16446194e-04]	km/s


**Algorithm 26** outlines the process for obtaining the topocentric right ascension and declination.

A rearranging of **Equation 4-3** gets us the slant-range and slant-range rate vectors:

$$
\begin{aligned}
\vec{\rho}_{ECI} = \vec{r}_{ECI} - \vec{r}_{SiteECI} \\
\dot{\vec{\rho}}_{ECI} = \vec{v}_{ECI} - \vec{v}_{SiteECI}
\end{aligned}
$$

The slant range is simply the magnitude of the vector, $\rho = \left| \vec{\rho}_{ECI} \right|$. The declination is found with:

$$
\sin(\delta_t) = \frac{\rho_K}{\rho}
$$

We can get the right ascension by implementing the following logic:

$$
\begin{cases}
\sqrt{\rho_I^2 + \rho_J^2} \neq 0: \quad
\displaystyle
\sin(\alpha_t) = \frac{\rho_J}{\sqrt{\rho_I^2 + \rho_J^2}} & \quad
\displaystyle
\cos(\alpha_t) = \frac{\rho_I}{\sqrt{\rho_I^2 + \rho_J^2}} \\
\sqrt{\rho_I^2 + \rho_J^2} = 0: \quad
\displaystyle
\sin(\alpha_t) = \frac{\dot{\rho}_J}{\sqrt{\dot{\rho}_I^2 + \dot{\rho}_J^2}} & \quad
\displaystyle
\cos(\alpha_t) = \frac{\dot{\rho}_I}{\sqrt{\dot{\rho}_I^2 + \dot{\rho}_J^2}} \quad
\end{cases}
$$

The rates can be determined with:

$$
\dot{\rho} = \frac{\vec{\rho}_{ECI} \cdot \dot{\vec{\rho}}_{ECI}}{\rho} \quad\quad
\dot{\alpha}_t = \frac{\dot{\rho}_I \rho_J - \dot{\rho}_J \rho_I}{-\rho_J^2 - \rho_I^2} \quad\quad
\dot{\delta}_t = \frac{\dot{\rho}_K - \dot{\rho} \sin(\delta_t)}{\sqrt{\rho_I^2 + \rho_J^2}} 
$$

Call the `rv2tradec` routine to obtain the topocentric right ascension and declination angles and angle rates:

In [11]:
rho, trtasc, tdecl, drho, dtrtasc, dtdecl = fc.rv2tradec(reci, veci, rseci, vseci)

print('Topocentric RA/Dec Values:\n')
print(f'range:\t\t\t{rho}\t\tkm')
print(f'right asc.:\t\t{np.degrees(trtasc)}\t\tdeg')
print(f'decl.:\t\t\t{np.degrees(tdecl)}\t\tdeg')
print(f'range rate:\t\t{drho}\t\tkm/s')
print(f'right asc. rate:\t{np.degrees(dtrtasc)}\t\tdeg/s')
print(f'decl. rate:\t\t{np.degrees(dtdecl)}\t\tdeg/s')

Topocentric RA/Dec Values:

range:			4437722626.693015		km
right asc.:		294.98911145219154		deg
decl.:			-20.823562340039164		deg
range rate:		-25.3606717651137		km/s
right asc. rate:	-1.2676744847943192e-07		deg/s
decl. rate:		-1.7108900667178414e-08		deg/s


Next, we want to find the azimuth and elevation angles, which requires converting the ECI state vectors to the ECEF frame:

In [12]:
aeci = [0, 0, 0]  # dummy acceleration, km/s²
recef, vecef, _ = eci2ecef(reci, veci, aeci, ttt, jdut1+jdut1frac, lod, xp, yp, ddpsi, ddeps, iau80arr)

print(f'recef:\t{recef}\tkm')
print(f'vecef:\t{vecef}\tkm/s')

recef:	[-2.93155946e+09 -2.93395314e+09 -1.57837994e+09]	km
vecef:	[-2.13936294e+05  2.13796180e+05  7.78610886e+00]	km/s


From there we implement the rest of **Algorithm 27**. The slant-range position and velocity vectors in ECEF are found with:

$$
\begin{aligned}
\vec{\rho}_{ECEF} &= \vec{r}_{ECEF} - \vec{r}_{SiteECEF} \\
\dot{\vec{\rho}}_{ECEF} &= \vec{v}_{ECEF}
\end{aligned}
$$

We then rotate the slant range to the topocentric-horizon (SEZ) system:

$$
\begin{aligned}
\vec{\rho}_{SEZ} = \left[ \text{ROT2}(90^{\circ} - \phi_{gd}) \right] \left[ \text{ROT3}(\lambda) \right] \ \vec{\rho}_{ECEF} \\
\dot{\vec{\rho}}_{SEZ} = \left[ \text{ROT2}(90^{\circ} - \phi_{gd}) \right] \left[ \text{ROT3}(\lambda) \right] \ \dot{\vec{\rho}}_{ECEF}
\end{aligned}
$$

Where $\rho = \left| \vec{\rho}_{SEZ} \right|$. The elevation angle is found with:

$$
\sin(el) = \frac{\rho_Z}{\rho}
$$

We can get the azimuth by implementing the following logic:

$$
\begin{cases}
\text{Elevation} \neq 90^{\circ}: \quad
\displaystyle
\sin(\beta) = \frac{\rho_E}{\sqrt{\rho_S^2 + \rho_E^2}} & \quad
\displaystyle
\cos(\beta) = \frac{-\rho_S}{\sqrt{\rho_S^2 + \rho_E^2}} \\
\text{Elevation} = 90^{\circ}: \quad
\displaystyle
\sin(\beta) = \frac{\dot{\rho}_E}{\sqrt{\dot{\rho}_S^2 + \dot{\rho}_E^2}} & \quad
\displaystyle
\cos(\beta) = \frac{\dot{-\rho}_S}{\sqrt{\dot{\rho}_S^2 + \dot{\rho}_E^2}} \quad
\end{cases}
$$

The rates can be determined with:

$$
\dot{\rho} = \frac{\vec{\rho}_{SEZ} \cdot \dot{\vec{\rho}}_{SEZ}}{\rho} \quad\quad
\dot{\beta} = \frac{\dot{\rho}_S \rho_E - \dot{\rho}_E \rho_S}{\rho_S^2 + \rho_E^2} \quad\quad
\dot{el} = \frac{\dot{\rho}_Z - \dot{\rho} \sin(el)}{\sqrt{\rho_S^2 + \rho_E^2}} 
$$

Call the `rv2razel` routine to obtain the azimuth and elevation angles and angle rates:

In [13]:
rho_hzn, az, el, drho_hzn, daz, del_el = fc.rv2razel(recef, vecef, latgd, lon, alt)

print('Horizon Az/El Values:')
print(f'range:\t\t{rho_hzn}\t\tkm')
print(f'az.:\t\t{np.degrees(np.mod(az, 2*np.pi))}\t\tdeg')
print(f'elev.:\t\t{np.degrees(el)}\t\tdeg')
print(f'range rate:\t{drho_hzn}\t\tkm/s')
print(f'az. rate:\t{np.degrees(daz)}\t\tdeg/s')
print(f'elev. rate:\t{np.degrees(del_el)}\t\tdeg/s')

Horizon Az/El Values:
range:		4437722626.693016		km
az.:		210.82506677475445		deg
elev.:		23.859505267041108		deg
range rate:	-25.360671765164735		km/s
az. rate:	0.0038629753442334575		deg/s
elev. rate:	-0.0016637109002570118		deg/s


Finally, we want to find the ecliptic latitude and longitude angles. The difference between the geocentric and heliocentric (ecliptic) systems is the obliquity of the ecliptic, $\epsilon$:

$$
\vec{r}_{XYZ} = \begin{bmatrix}
r\cos(\phi_{ecliptic}) \cos(\lambda_{ecliptic}) \\
r\cos(\phi_{ecliptic}) \sin(\lambda_{ecliptic}) \\
r\sin(\phi_{ecliptic})
\end{bmatrix} \quad\quad
\vec{r}_{IJK} = \text{ROT1} \left[ -\epsilon \right] \ \vec{r}_{XYZ}
$$

Or, using **Equation 4-15**:

$$
\vec{r}_{XYZ} =
r \begin{bmatrix}
\cos(\phi_{ecliptic}) \cos(\lambda_{ecliptic}) \\
\cos(\epsilon) \cos(\phi_{ecliptic}) \sin(\lambda_{ecliptic}) - \sin(\epsilon) \sin(\phi_{ecliptic}) \\
\sin(\epsilon) \cos(\phi_{ecliptic}) \sin(\lambda_{ecliptic}) + \cos(\epsilon) \sin(\phi_{ecliptic}) \\
\end{bmatrix} \\
$$

Although the textbook doesn’t explicitly provide equations for the rates of change of ecliptic latitude and longitude, they can be computed in the same spirit as the previous conversions, which are calculated using vector derivatives consistent with the position and velocity transformations.

Call the `rv2ell` routine to obtain the ecliptic latitude and longitude angles and angle rates:

In [14]:
rho_ecl, ecllon, ecllat, drho_ecl, decllon, decllat = fc.rv2ell(reci, veci)

print('Ecliptic Lat/Lon Values:')
print(f'range:\t\t{rho_ecl}\t\t\tkm')
print(f'lon.:\t\t{np.degrees(np.mod(ecllon, 2*np.pi))}\t\tdeg')
print(f'lat.:\t\t{np.degrees(ecllat)}\t\tdeg')
print(f'range rate:\t{drho_ecl}\t\tkm/s')
print(f'lon. rate:\t{np.degrees(decllon)}\t\tdeg/s')
print(f'elev. rate:\t{np.degrees(decllat)}\t\tdeg/s')

Ecliptic Lat/Lon Values:
range:		4437725220.51			km
lon.:		293.2582108295137		deg
lat.:		0.6207506500195128		deg
range rate:	-25.530330940000006		km/s
lon. rate:	-1.1583629035019951e-07		deg/s
elev. rate:	1.5470846322574515e-09		deg/s


## Summary
---

The table below summarizes the results from all the conversions. We use the terms *lateral* for right ascension, azimuth, and longitude; and *vertical* for declination, elevation, and latitude:

In [15]:
summary_data = {
    "System": ["Geocentric", "Topocentric", "Horizon", "Ecliptic"],
    "Range (km)": [rr, rho, rho_hzn, rho_ecl],
    "Lateral (°)": np.degrees([rtasc, trtasc, np.mod(az, 2*np.pi), np.mod(ecllon, 2*np.pi)]),
    "Vertical (°)": np.degrees([decl, tdecl, el, ecllat]),
    "Range Rate (km/s)": [drr, drho, drho_hzn, drho_ecl],
    "Lateral Rate (°/s)": np.degrees([drtasc, dtrtasc, daz, decllon]),
    "Vertical Rate (°/s)": np.degrees([ddecl, dtdecl, del_el, decllat]),
}

format_dict = {
    "Range (km)": "{:.3f}",
    "Lateral (°)": "{:.8f}",
    "Vertical (°)": "{:.8f}",
    "Range Rate (km/s)": "{:.8f}",
    "Lateral Rate (°/s)": "{:.12f}",
    "Vertical Rate (°/s)": "{:.12f}",
}

df = pd.DataFrame(summary_data)
df.style.format(format_dict).hide(axis="index")

System,Range (km),Lateral (°),Vertical (°),Range Rate (km/s),Lateral Rate (°/s),Vertical Rate (°/s)
Geocentric,4437725220.510,294.98914580,-20.82349440,-25.53033094,-0.000000122440,-0.000000017940
Topocentric,4437722626.693,294.98911145,-20.82356234,-25.36067177,-0.000000126767,-0.000000017109
Horizon,4437722626.693,210.82506677,23.85950527,-25.36067177,0.003862975344,-0.001663710900
Ecliptic,4437725220.510,293.25821083,0.62075065,-25.53033094,-0.000000115836,0.000000001547


The textbook example recalculates the entire problem assuming a satellite at a range of 12,756 km with a velocity of 6.798614 km/s to illustrate more dramatic differences between the resulting values, since these results show less variation between the different representations